In [1]:
import esm
import pyvista as pv
import os
import ladybug as lb
import pandas as pd
import subprocess
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np

cwd = os.getcwd()
src = os.path.dirname(cwd)

Path to radiance is set to: /usr/local/radiance
Path to perl is set to: /usr/bin


In [2]:
# import all external geometry with pyvista
wal_filepath = os.path.join(src, "data", "test_room_walls_02.obj")
wal_mesh = pv.read(wal_filepath)
fen_filepath = os.path.join(src, "data", "test_room_fens_02.obj")
fen_mesh = pv.read(fen_filepath)

con_buildings_filepath = os.path.join(src, "data", "test_context_01_buildings.obj")
con_buildings_mesh = pv.read(con_buildings_filepath)
con_ground_filepath = os.path.join(src, "data", "test_context_01_ground.obj")
con_ground_mesh = pv.read(con_ground_filepath)
# con_sky_filepath = os.path.join(src, "data", "test_context_01_sky.obj")
# con_sky_mesh = pv.read(con_sky_filepath)
con_vegetation_filepath = os.path.join(src, "data", "test_context_01_vegetation.obj")
con_vegetation_mesh = pv.read(con_vegetation_filepath)

# construct honeybee surfaces from the mesh
wal_hb_surf = esm.pv_mesh_to_hbsurface(wal_mesh, 0, "wall", esm.material_plastic)
fen_hb_surf = esm.pv_mesh_to_hbsurface(fen_mesh, 5, "window", esm.material_glass)
con_buildings_hb_surf = esm.pv_mesh_to_hbsurface(con_buildings_mesh, 0, "context_buildings", esm.material_plastic)
con_ground_hb_surf = esm.pv_mesh_to_hbsurface(con_ground_mesh, 0, "context_ground", esm.material_plastic)
# con_sky_hb_surf = esm.pv_mesh_to_hbsurface(con_sky_mesh, 0, "context_sky", esm.material_plastic)
con_vegetation_hb_surf = esm.pv_mesh_to_hbsurface(con_vegetation_mesh, 0, "context_vegetation", esm.material_plastic)

hb_surfaces = wal_hb_surf + fen_hb_surf + con_buildings_hb_surf + con_ground_hb_surf + con_vegetation_hb_surf

In [3]:
# generate grid of test points
test_pts = [(1, 1, 1),(6, 1, 1),(1, 6, 1),(1, 1, 6)]
test_vec = [(0, 0, 1),(0, 0, 1),(0, 0, 1),(0, 0, 1)]
analysis_grid = esm.AnalysisGrid.from_points_and_vectors(test_pts, test_vec, 'test_grid')

# # generate the sky
# sky = esm.CertainIlluminanceLevel(illuminance_value=10000,sky_type=1)

# put the recipe together
rp = esm.ContextViewGridBased(analysis_grids=(analysis_grid,),
               hb_objects=hb_surfaces)

Found 5486 opaque surfaces.
Found 1 fenestration surfaces.
Found 0 window-groups.


In [4]:
# write simulation to folder
project_name = 'clean_context_view_test'
batch_file = rp.write(
    target_folder='.', project_name=project_name)


Writing recipe contents to: ./clean_context_view_test/gridbased


In [5]:
# run the simulation
rp.run(batch_file, debug=False)

True

In [7]:
# # results - in this case it will be an analysis grid
# result = rp.results()[0]

# # extract the points of interest from the recipe
# result_points = [[p[0], p[1], p[2]] for p in result.points]

Unloading the current values from the analysis grids.


In [10]:
rs_path = os.path.join(project_name, 'gridbased', 'rtrace_res.txt')
rtrace_res = pd.read_csv(rs_path, skiprows=8, sep='\t', header=None)
rtrace_res

,0,1,2,3,4,5,6
0,0.0,0.0,0.0,wall,1.902113,1.902113,NaN
1,0.0,0.0,0.0,context_buildings,4.205226,4.205226,NaN
2,0.0,0.0,0.0,wall,1.175571,1.175571,NaN
3,0.0,0.0,0.0,window,119.185700,1.175571,NaN
4,0.0,0.0,0.0,wall,1.902113,1.902113,NaN
...,...,...,...,...,...,...,...
637,0.0,0.0,0.0,wall,1.012465,1.012465,NaN
638,0.0,0.0,0.0,wall,1.091412,1.091412,NaN
639,0.0,0.0,0.0,wall,1.095311,1.095311,NaN
640,0.0,0.0,0.0,wall,1.034776,1.034776,NaN


In [11]:
pnt_cnt = 1 # grid point count
labels = np.array(rtrace_res[3]) # .reshape(pnt_cnt, -1)
dist_0 = np.array(rtrace_res[4]) # .reshape(pnt_cnt, -1)

window_mask = labels=='window'
win_hit_count = np.sum(window_mask.reshape(pnt_cnt, -1), axis=1)

dist_window = dist_0
dist_window[1 - window_mask] = np.nan
min_win_dist = np.min(dist_window.reshape(pnt_cnt, -1), axis=1)
print(min_win_dist)
sum_win_dist = np.sum(dist_window.reshape(pnt_cnt, -1), axis=1)
print(sum_win_dist)
var_win_dist = np.var(dist_window.reshape(pnt_cnt, -1), axis=1)
print(var_win_dist)

[nan]
[nan]
[nan]


In [12]:
# distances outside the window

labels_array = rtrace_res[3]
window_dist_array = rtrace_res[4].values.reshape(int(view.x_resolution), int(view.y_resolution))
window_dist_masked = np.ma.masked_where((labels_array!='window'), rtrace_res[4]).reshape(int(view.x_resolution), int(view.y_resolution))

plt.imshow(window_dist_masked, origin='upper', norm=LogNorm(vmin=0.01, vmax=1e10))
plt.colorbar()
plt.show()

print(window_dist_masked.min(), window_dist_masked.max())

NameError: name 'view' is not defined

In [13]:
# Distances and angles on the central row (eye level)

cx, cy = int(int(view.x_resolution)/2), int(int(view.y_resolution)/2)

c_row = labels_array.iloc[int(view.x_resolution)*(cy-1):int(view.x_resolution)*(cy)]
window_h_aperture_angle = c_row.groupby(c_row).count()['window'] * 180 / int(view.x_resolution)
window_h_aperture_angle

NameError: name 'view' is not defined

In [ ]:
# distances within the room

dist_array = rtrace_res[5].values.reshape(int(view.x_resolution),int(view.y_resolution))
dist_c = dist_array[cx, cy]

plt.imshow(dist_array, origin='upper')
plt.colorbar()
plt.show()

print(dist_array.min(), dist_array.max(), dist_c)